Hi, I'm Soma! You can find me on email at [jonathan.soma@gmail.com](mailto:jonathan.soma@gmail.com), on Twitter at [@dangerscarf](https://twitter.com/dangerscarf), or maybe even on [this newsletter I've never sent](https://tinyletter.com/jsoma).

# The secret world of undocumented APIs

While scraping a website is all good and fun, sometimes there's a better/faster/cheaper way to get your data: **undocumented APIs!**

An API is how two computers talk to each other without the ugliness of the web getting involved. To be overly simplistic, instead of browsing a site like a normal human being, your computer visits a special URL to search or download data in bulk. For example, Twitter's API is how researchers got millions and millions of tweets before Musk locked 'em all out.

While many APIs are public-facing and advertised, some are semi-secret or unofficial. For example, when you visit [Pitchfork](https://pitchfork.com) and search for music reviews, your browser secretly visits [this URL](https://pitchfork.com/api/v2/search/?genre=experimental&genre=global&genre=jazz&genre=metal&genre=pop&genre=rap&genre=rock&types=reviews&sort=publishdate%20desc%2Cposition%20asc&size=5&start=0&rating_from=0.0) to find the data that it eventually displays on the page. That listing of data – the API "endpoint" – is all about computers reading it, not people, so it looks awful and ugly and very much like this:

```python
{"count":10000,"previous":null,"next":null,"results":{"category":null,"list":[{"dek":"<p>The pop trio’s new single casts the iconic 24-hour breakfast chain as a place of conversation and healing, not fights and thrown chairs.</p>\n","seoDescription":"The pop trio’s new single casts the iconic 24-hour breakfast chain as a place of conversation and healing, not fights and thrown chairs.","promoDescription":"<p>The pop trio’s new single casts the iconic 24-hour breakfast chain as a place of conversation and healing, not fights and thrown chairs.</p>\n","socialDescription":"The pop trio’s new single casts the iconic 24-hour breakfast chain as a place of conversation and healing, not fights and thrown chairs.","authors":[{"id":"592604b57fd06e5349102f43","name":"Evan Minsker","title":"Associate News Director","url":"/staff/evan-minsker/","slug":"staff/evan-minsker"}]
```

Horrifying, right? In this tutorial, we'll be looking at three things:

1. Exploring how LangChain talks to APIs
2. Using ChatGPT to document previously-undocumented APIs
3. Comparing two techniques for enabling LangChain to use undocumented APIs (explicit vs implicit documentation)

By the end we'll have developed a tool that can use the unofficial Pitchfork API to answer natural-language questions about albums they've reviewed.

::: {.callout-note appearance='simple'}

Want to learn how to discover undocumented APIs? Check out [Inspect Element](https://inspectelement.org/apis.html) by [Leon Yin](https://twitter.com/leonYin/)!

:::


## Setup

Give me one moment to set things up! First we'll pull in all of our API keys using [python-dotenv](https://pypi.org/project/python-dotenv/), then we'll use a thousand and one imports to bring in LangChain and friends.

In [1]:
%load_ext dotenv
%dotenv

In [144]:
# LangChain imports
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.chains import APIChain
from langchain.prompts.prompt import PromptTemplate
from langchain.callbacks import get_openai_callback
from langchain import LLMChain

# Normal human imports
import requests
from bs4 import BeautifulSoup
import json

Even though GPT-4 is out, we'll be using GPT-3.5-turbo for this one. At the moment it's infinitely cheaper, and every penny matters!

In [113]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

# LangChain and documented APIs

[LangChain](https://python.langchain.com) is a "framework for developing applications powered by language models," which really undersells the fact that it's a universe-altering suite of tools for putting GPT (and related tools) to work. In this tutorial we're focusing on how it **interacts with APIs.**

In the [LangChain documentation for working with APIs](https://python.langchain.com/en/latest/modules/chains/examples/api.html) there's a super-simple example of using `APIChain` to get an answer from a [free weather API](https://open-meteo.com/). You create your API-aware "chain" from two things: your large language model (in this case, GPT-3.5-turbo) and the documentation to the API.

In [ ]:
chain = APIChain.from_llm_and_api_docs(llm, open_meteo_docs.OPEN_METEO_DOCS, verbose=True)

Once your chain is created, you ask it your question and the chain goes to work sending information back and forth with ChatGPT:

1. The chain sends the API docs to ChatGPT, asking what API URL you should visit
2. ChatGPT reads the docs, sends back a URL
2. LangChain obtains the data from the URL
3. The data is sent back to ChatGPT, which uses it to answer your question

Let's see it in action:

In [114]:
chain.run('What is the weather like right now in Munich, Germany in degrees Farenheit?')



> Entering new APIChain chain...
https://api.open-meteo.com/v1/forecast?latitude=48.137154&longitude=11.576124&current_weather=true&temperature_unit=fahrenheit
{"latitude":48.14,"longitude":11.58,"generationtime_ms":0.15795230865478516,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":526.0,"current_weather":{"temperature":40.2,"windspeed":8.8,"winddirection":261.0,"weathercode":2,"is_day":0,"time":"2023-04-08T00:00"}}

> Finished chain.


'The current weather in Munich, Germany is 40.2 degrees Fahrenheit.'

You can see each step of the process: the URL, the data, the answer! This transparency is thanks to us using `verbose=True` when we first created the chain.

The "secret sauce" of the `APIChain` formula is the OpenMeteo API documentation. The documentation is detailed enough to provide ChatGPT with everything it needs to know in creating the API URL: from what I can see, it needs a latitude and longitude, a `current_weather=true`, and a demand to be in degrees fahrenheit.

How detailed is the OpenMeteo documentation that ChatGPT is using? It's easy enough to examine the documentation for this particular API, as it's actually provided as part of LangChain:

In [115]:
print(open_meteo_docs.OPEN_METEO_DOCS)

BASE URL: https://api.open-meteo.com/

API Documentation
The API endpoint /v1/forecast accepts a geographical coordinate, a list of weather variables and responds with a JSON hourly weather forecast for 7 days. Time always starts at 0:00 today and contains 168 hours. All URL parameters are listed below:

Parameter	Format	Required	Default	Description
latitude, longitude	Floating point	Yes		Geographical WGS84 coordinate of the location
hourly	String array	No		A list of weather variables which should be returned. Values can be comma separated, or multiple &hourly= parameter in the URL can be used.
daily	String array	No		A list of daily weather variable aggregations which should be returned. Values can be comma separated, or multiple &daily= parameter in the URL can be used. If daily weather variables are specified, parameter timezone is required.
current_weather	Bool	No	false	Include current weather conditions in the JSON output.
temperature_unit	String	No	celsius	If fahrenheit is set, al

Look at all those details and options! It might be overwhelming to us, but ChatGPT has no problem figuring it out.

If we want to use the [Pitchfork API](https://pitchfork.com/api/v2/search/?genre=experimental&genre=global&genre=jazz&genre=metal&genre=pop&genre=rap&genre=rock&types=reviews&sort=publishdate%20desc%2Cposition%20asc&size=5&start=0&rating_from=0.0) to ask questions in a similar fashion, it seems like we might need some documentation for it.

There's only one problem: **it's an unofficial, undocumented API.**

# Automatic documentation generation

Luckily for us, APIs aren't (often) all that complicated. Let's look at the Pitchfork API's URL:

```
https://pitchfork.com/api/v2/search/?genre=experimental&genre=global&genre=jazz&genre=metal&genre=pop&genre=rap&genre=rock&types=reviews&sort=publishdate%20desc%2Cposition%20asc&size=5&start=0&rating_from=0.0
```

After the `/search/` part, we see a lot of things we can making assumptions about.

* `size=5` probably has to do with how many results are returned. In this case, our search gives us 5 results.
* `genre=jazz&genre=metal` is probably all of the genres of albums to return
* `rating_from=0.0` potentially sets a lower bar for the rating of the album reviews. Pitchfork ranks them 0-10, so this includes all albums.
* There's also `types`, `sort`, `start`... We can guess about those, too!

Do we know that our guesses are correct? Absolutely not. But with a little time and some manual labor, we might be able to test our hypotheses about what each parameter actually means!

But we have neither time or manual labor: we're playing loose and fast with the truth, we're moving fast and breaking things, we're fucking around and hopefully only finding out beautiful, blissful things. Instead, we have AI.

Let's just irresponsibly ask ChatGPT to make all of those assumptions for us, and write some nice beautiful documentation just like the OpenMeteo ones!

Down below we build a prompt that takes an API url and asks ChatGPT to write us "detailed documentation" for the provided URL.

In [136]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["api_url"],
    template="""Act as a technical writer. Write detailed documentation for the API that exists at {api_url}. Only detail the request, do not describe the response. Do not include any parameters not in the sample endpoint."""
)

chain = LLMChain(
    llm=llm,
    verbose=True,
    prompt=prompt
)

The code is slightly more complicated than I'd like, as a few weeks ago LangChain decided that chat-based LLMs like GPT-3.5 needed to be talked to with prompts instead of just saying `llm.run`. I'm sorry it's a little verbose, but we'll both survive.

Notice my extra-stern warning to not provide an example response. While we're fine with ChatGPT inventing how to use it, we'd rather not have it waste time inventing data that comes back from the API exactly looks like.

Now we'll take our Pitchfork API url and feed it into the chain.

In [137]:
url = "https://pitchfork.com/api/v2/search/?genre=experimental&genre=global&genre=jazz&genre=metal&genre=pop&genre=rap&genre=rock&types=reviews&sort=publishdate%20desc%2Cposition%20asc&size=5&start=0&rating_from=0.0"

response = chain.run(url)
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
Act as a technical writer. Write detailed documentation for the API that exists at https://pitchfork.com/api/v2/search/?genre=experimental&genre=global&genre=jazz&genre=metal&genre=pop&genre=rap&genre=rock&types=reviews&sort=publishdate%20desc%2Cposition%20asc&size=5&start=0&rating_from=0.0. Only detail the request, do not describe the response. Do not include any parameters not in the sample endpoint.

> Finished chain.
API Documentation

Pitchfork Search API

This API is used to search for reviews on Pitchfork based on specified genres, types, sorting, and size.

Sample Endpoint:

https://pitchfork.com/api/v2/search/?genre=experimental&genre=global&genre=jazz&genre=metal&genre=pop&genre=rap&genre=rock&types=reviews&sort=publishdate%20desc%2Cposition%20asc&size=5&start=0&rating_from=0.0

Request Method: GET

Request Parameters:

• genre: This parameter is used to specify the genre of the review. You can specify multiple genre

That documentation is far nicer than anything I'd personally write!

Is it all correct? **We have no idea!** But to continue thinking positively: if we found a bug we can always make manual edits.

Now let's talk about how we want to use it.

# Explicit documentation

I'm going to call the type of text above **explicit documentation**. It's "real" documentation, words phrased and formatted in order to communicate specific details about and examples of the API.

This explicit documentation is just like the OpenMeteo documentation from the LangChain documentation, and we're going to use it in the exact same way. We'll make a new `APIChain`, giving it the language model and the API docs.

After the chain is made, we'll ask it our question: it should use the documentation to format a URL, then use the data from the URL to answer the question.

In [138]:
# Save the response from above as `explicit_docs`, to use later
explicit_docs = response
explicit_chain = APIChain.from_llm_and_api_docs(llm, explicit_docs, verbose=True)

In [139]:
response = explicit_chain.run("What was the first rap album reviewed by pitchfork?")
print(response)



> Entering new APIChain chain...
https://pitchfork.com/api/v2/search/?genre=rap&types=reviews&sort=publishdate%20asc&size=1&rating_from=0.0
{"count":4253,"previous":null,"next":null,"results":{"category":null,"list":[{"tombstone":{"bnm":false,"bnr":false,"albums":[{"id":"5929c3a7eb335119a49ed773","album":{"artists":[{"id":"592994259d034d5c69bf1739","display_name":"Roots Manuva","url":"/artists/2672-roots-manuva/","genres":[{"display_name":"Electronic","slug":"electronic"},{"display_name":"Jazz","slug":"jazz"},{"display_name":"Rap","slug":"rap"}],"slug":"592994259d034d5c69bf1739","photos":{"tout":{"width":300,"height":300,"credit":"","caption":"","altText":"Image may contain: Face, Human, Person, Roots Manuva, Head, Photo, Portrait, and Photography","modelName":"photo","title":"Roots Manuva artist image","sizes":{"sm":"https://media.pitchfork.com/photos/59299426c0084474cd0bec29/1:1/w_150/3d81e0d6.jpg","m":"https://media.pitchfork.com/photos/59299426c0084474cd0bec29/1:1/w_300/3d81e0d6.

The URL it chose to visit was 

```
https://pitchfork.com/api/v2/search/?genre=rap&types=reviews&sort=publishdate%20asc&size=1&rating_from=0.0
```

It adjusted the genre list, the publish date, and decided it only needed a single result! That seems pretty remarkable to me, and the result of an album from 1999 also seems reasonable.

# Implicit documentation

While the explicit documentation above is pretty fantastic, **it also might be a waste of time.** Think about it: if ChatGPT generates a new URL by reading the documentation it created by reading the single URL...

```{mermaid}
flowchart LR
  A[URL] --> B[Documentation]
  B --> C[New URL]
```

...why can't we just cut out the middleman? Can't we just say, "here's a sample URL, figure out the new one?"

```{mermaid}
flowchart LR
  A[URL] -.- B[Documentation]
  B -.- C[New URL]
  A ==> C
```

The documentation isn't providing anything it doesn't know already, so it seems reasonable, right? Let's try it now! Now we're going to create **implicit docs**, which briefly describe the existence of the API and give a sample endpoint. Instead of all that detail, it's just the one URL!

In [140]:
implicit_docs = """
Pitchfork has an API with a sample endpoint at https://pitchfork.com/api/v2/search/?genre=experimental&genre=global&genre=jazz&genre=metal&genre=pop&genre=rap&genre=rock&types=reviews&sort=publishdate%20desc%2Cposition%20asc&size=5&start=0&rating_from=0.0
"""

implicit_chain = APIChain.from_llm_and_api_docs(llm, implicit_docs, verbose=True)

**What are you expecting?** Let's give it a shot.

In [141]:
implicit_chain.run("What was the first rap album reviewed by pitchfork?")



> Entering new APIChain chain...
https://pitchfork.com/api/v2/search/?genre=rap&types=reviews&sort=publishdate%20asc&size=1&start=0&rating_from=0.0
{"count":4253,"previous":null,"next":null,"results":{"category":null,"list":[{"tombstone":{"bnm":false,"bnr":false,"albums":[{"id":"5929c3a7eb335119a49ed773","album":{"artists":[{"id":"592994259d034d5c69bf1739","display_name":"Roots Manuva","url":"/artists/2672-roots-manuva/","genres":[{"display_name":"Electronic","slug":"electronic"},{"display_name":"Jazz","slug":"jazz"},{"display_name":"Rap","slug":"rap"}],"slug":"592994259d034d5c69bf1739","photos":{"tout":{"width":300,"height":300,"credit":"","caption":"","altText":"Image may contain: Face, Human, Person, Roots Manuva, Head, Photo, Portrait, and Photography","modelName":"photo","title":"Roots Manuva artist image","sizes":{"sm":"https://media.pitchfork.com/photos/59299426c0084474cd0bec29/1:1/w_150/3d81e0d6.jpg","m":"https://media.pitchfork.com/photos/59299426c0084474cd0bec29/1:1/w_300/3

'The first rap album reviewed by Pitchfork was "Brand New Secondhand" by Roots Manuva, published on March 23, 1999, with a rating of 9.5. The API call used to retrieve this information was: https://pitchfork.com/api/v2/search/?genre=rap&types=reviews&sort=publishdate%20asc&size=1&start=0&rating_from=0.0.'

That's an A+ perfect response, with a heck of a lot less work.

# Summary and differences

There are two major ways the explicit and implicit approaches differ: success rates and costs.

## Success rates

Believe it or not, **the implicit approach has a much higher success rate!**

In writing this piece, I've had to tweak the prompt that generates the explicit documentation again and again. ChatGPT seems to go out of its way to lie about the API, and not even in subtle ways:

* It loves to introduce new, non-existent features
* Turn the `genre=rap&genre=folk` parameter into various flavors of `genre=rap,folk`, and just generally sacrifices the reality of the API for 
* Invent lower and upper bounds for page sizes and ratings

While these might be good architectural changes or useful new features, they absolutely aren't implied by the original URL! Without adding a push to be conservative to the prompt, we end up with docs that are completely misleading.

When you take the misleading documentation and feed it to the `APIChain` prompt, it winds up screwing up the request a large portion of the time. Overall I've found it breaks about a third of the time on the explicitly-documented example!

On the other hand, explicit documentation allows you to clean up and customize the docs. If you find out the maximum and minimum page size, you're free to add it! If other genres get released or removed you're more than welcome to edit the list.

But if you're lazy, and just looking for a shortcut? **Implicit docs always peform better**.


## Costs

While we're all very excited about using the various OpenAI APIs, **they do cost money.** Yes, `GPT-3.5-turbo` is remarkably inexpensive compared to its peers, but we aren't here to waste money! If we can keep the prompt smaller our queries cost less, and saving money is the second-quickest route to happiness.

Let's use LangChain's `get_openai_callback` to examine the token count and cost of our explicit vs implicit requests. Note that we're using `verbose=False` here to reduce clutter.

In [142]:
explicit_chain = APIChain.from_llm_and_api_docs(llm, explicit_docs, verbose=False)

with get_openai_callback() as cb:
    response = explicit_chain.run("What was the first rap album reviewed by pitchfork?")
    print(f"Response: {response}")
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")

Response: The first rap album reviewed by Pitchfork was Roots Manuva's "Brand New Secondhand" released in 1999, with a review published on March 23, 1999, with a rating of 9.5.
Total Tokens: 3058
Prompt Tokens: 2970
Completion Tokens: 88
Total Cost (USD): $0.006116


In [143]:
implicit_chain = APIChain.from_llm_and_api_docs(llm, implicit_docs, verbose=False)

with get_openai_callback() as cb:
    response = implicit_chain.run("What was the first rap album reviewed by pitchfork?")
    print(f"Response: {response}")
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")

Response: The first rap album reviewed by Pitchfork was "Brand New Secondhand" by Roots Manuva, with a rating of 9.5. The review was written by Paul Cooper and was published on March 23, 1999.
Total Tokens: 1811
Prompt Tokens: 1722
Completion Tokens: 89
Total Cost (USD): $0.0036220000000000002


Along with being consistent less correct, **the explicit chain costs around twice as much!** Just above 0.6 cents for explicit compared to `0.36` for implicit. Depending on how wordy GPT decides to be, some of my tests have seen it up to tree times as much!

The wildest part about this large difference is that both queries almost always making the *same* API response, which I assume would take up the bulk of the tokens (secret fact: Pitchfork's API is so wordy I changed the example to `size=5` so that it would fit in the GPT-3.5-turbo context window).

## The takeaway

Unless you enjoy editing or spending money, it looks like *less is more*.